In [ ]:
!pip install gcsfs

     |████████████████████████████████| 112kB 34.5MB/s 
     |████████████████████████████████| 1.3MB 25.5MB/s 
     |████████████████████████████████| 296kB 40.5MB/s 
     |████████████████████████████████| 143kB 49.3MB/s 


In [ ]:
import os
from google.cloud import bigquery as bq
from google.cloud import storage
from google.colab import auth
import csv
import pandas as pd
import string
import gcsfs
import calendar
import datetime as dt

In [ ]:
auth.authenticate_user()

project_id = 'citp-sm-reactions'
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
os.environ["GCLOUD_PROJECT"] = "citp-sm-reactions"
table_id = "citp-sm-reactions.reddit_clean_comments.reaction_interventions_epoc"

bq_client = bq.Client()



In [ ]:
job_config = bq.LoadJobConfig(
    schema=[
        bq.SchemaField("subreddit", bq.enums.SqlTypeNames.STRING),
        bq.SchemaField("intervention", bq.enums.SqlTypeNames.STRING),
        bq.SchemaField("start_date", bq.enums.SqlTypeNames.TIMESTAMP),
        bq.SchemaField("end_date", bq.enums.SqlTypeNames.TIMESTAMP),
    ],
)


In [ ]:
os.chdir('/content/gdrive/MyDrive/reddit/etl')

In [ ]:
intervention_df=pd.read_csv('reddit_interventions.csv')[['subreddit', 'intervention', 'start_date', 'end_date']]
intervention_df['start_date']=pd.to_datetime(intervention_df['start_date'])
intervention_df['end_date']=pd.to_datetime(intervention_df['end_date'])

intervention_df['start_date_unix_epoc']=((intervention_df['start_date'] - dt.datetime(1970,1,1)).dt.total_seconds()).astype(int)
intervention_df['end_date_unix_epoc']=((intervention_df['end_date'] - dt.datetime(1970,1,1)).dt.total_seconds()).astype(int)
intervention_df.head()

,subreddit,intervention,start_date,end_date,start_date_unix_epoc,end_date_unix_epoc
0,Conservative,baseline,2012-12-31,2013-10-31,1356912000,1383177600
1,Conservative,upvote_only,2013-11-01,2014-06-30,1383264000,1404086400
2,Conservative,no_votes,2014-07-01,2018-01-01,1404172800,1514764800
3,exmuslim,baseline,2010-01-01,2011-07-31,1262304000,1312070400
4,exmuslim,upvote_only,2011-08-01,2018-01-01,1312156800,1514764800


Timestamp('2012-12-31 00:00:00')

In [ ]:
job = bq_client.load_table_from_dataframe(intervention_df, table_id, job_config=job_config)  # Make an API request.
job.result() 

table = bq_client.get_table(table_id)  # Make an API request.
print("Loaded {} rows and {} columns to {}".format(
                table.num_rows, len(table.schema), table_id
            )
        )

Loaded 30 rows and 6 columns to citp-sm-reactions.reddit_clean_comments.reaction_interventions_epoc
